In [1]:
import torch
import torch.nn as nn
import torchvision
import torchvision.transforms as transforms

# Device configuration
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [2]:
# Hyper-parameters
num_epochs = 5
num_classes = 10
batch_size = 100
learning_rate = 0.001

# MNIST dataset
train_dataset = torchvision.datasets.MNIST(root='../data', 
                                           transform=transforms.ToTensor(), 
                                           download=True, 
                                           train=True)

test_dataset = torchvision.datasets.MNIST(root='../data', 
                                          transform=transforms.ToTensor(), 
                                          train=False)

# Data loader
train_loader = torch.utils.data.DataLoader(dataset=train_dataset, 
                                           batch_size=batch_size, 
                                           shuffle=True)

test_loader = torch.utils.data.DataLoader(dataset=test_dataset, 
                                          batch_size=batch_size, 
                                          shuffle=False)

# Convolutional neural network (two convolutional layers)
class ConvNet(nn.Module):
    def __init__(self, num_classes=10):
        super(ConvNet, self).__init__()
        # Conv1d: vector; Conv2d: matrix; Conv3d: tensor. The kernels are different in rank.
        # nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0, dilation=1, groups=1, bias=True)
        # Since the MNIST dataset is in greyscale, the input channel is 1
        # BatchNorm2d: apply batch normalization
        # Batch Normalization: https://zhuanlan.zhihu.com/p/24810318
        # Compute output size of a layer: (input_size + 2*padding - kernel_size)/stride + 1
        self.layer1 = nn.Sequential(
            nn.Conv2d(1, 16, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(16), 
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )        
        self.layer2 = nn.Sequential(
            nn.Conv2d(16, 32, kernel_size=5, stride=1, padding=2),
            nn.BatchNorm2d(32),
            nn.ReLU(),
            nn.MaxPool2d(kernel_size=2, stride=2)
        )
        self.fc = nn.Linear(7*7*32, num_classes)  # The output size of the layers 2 is 7*7*32
        
    def forward(self, x):
        out = self.layer1(x)
        out = self.layer2(out)  # size: [N, out_channel, out_h, out_h]
#         print(out.size())
        out = out.reshape(out.size(0), -1)  # size: [N, out_channel * out_h * out_w]
        out = self.fc(out)
        return out
    
model = ConvNet(num_classes).to(device)

# Loss and optimize
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

In [3]:
# Train the model
total_step = len(train_loader)
for epoch in range(num_epochs):
    for i, (images, labels) in enumerate(train_loader):
        images = images.to(device)
        labels = labels.to(device)
        
        # Forward pass
        outputs = model(images)
        loss = criterion(outputs, labels)
        
        # Backward and optimize
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        if (i+1) % 100 == 0:
            print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}'
                  .format(epoch+1, num_epochs, i+1, total_step, loss.item()))

Epoch [1/5], Step [100/600], Loss: 0.1411
Epoch [1/5], Step [200/600], Loss: 0.1978
Epoch [1/5], Step [300/600], Loss: 0.0842
Epoch [1/5], Step [400/600], Loss: 0.0115
Epoch [1/5], Step [500/600], Loss: 0.0288
Epoch [1/5], Step [600/600], Loss: 0.0369
Epoch [2/5], Step [100/600], Loss: 0.0407
Epoch [2/5], Step [200/600], Loss: 0.0484
Epoch [2/5], Step [300/600], Loss: 0.0677
Epoch [2/5], Step [400/600], Loss: 0.0773
Epoch [2/5], Step [500/600], Loss: 0.0620
Epoch [2/5], Step [600/600], Loss: 0.0118
Epoch [3/5], Step [100/600], Loss: 0.0153
Epoch [3/5], Step [200/600], Loss: 0.0638
Epoch [3/5], Step [300/600], Loss: 0.0453
Epoch [3/5], Step [400/600], Loss: 0.0308
Epoch [3/5], Step [500/600], Loss: 0.0289
Epoch [3/5], Step [600/600], Loss: 0.0149
Epoch [4/5], Step [100/600], Loss: 0.0144
Epoch [4/5], Step [200/600], Loss: 0.0046
Epoch [4/5], Step [300/600], Loss: 0.2360
Epoch [4/5], Step [400/600], Loss: 0.0198
Epoch [4/5], Step [500/600], Loss: 0.0306
Epoch [4/5], Step [600/600], Loss:

In [4]:
# Test the model
with torch.no_grad():
    correct = 0
    total = 0
    for images, labels in test_loader:
        images = images.to(device)
        labels = labels.to(device)
        outputs = model(images)
        _, predicted = torch.max(outputs.data, 1)
        total += len(labels)
        correct += (predicted == labels).sum().item()
        
    print('Test Accuracy of the model on the 10000 test images: {}%'.format(correct / total * 100))
    
# Save the model checkpoint
torch.save(model.state_dict(), 'model.ckpt')

Test Accuracy of the model on the 10000 test images: 99.06%
